## Example: Sequence Data

Next, let's consider the AAV dataset, designed vs mutant split, 
from the FLIP benchmark suite. For this dataset, we train on 200,000
length 57 amino acid sequences and try to predict the fitness
of a pre-specified test set. Dallago et al. report that a standard
1d-CNN trained on this achieves a Spearman's r of 0.75, while
a 750-million parameter pretrained model that took 50 GPU-days of
time to train achieves Spearman's r of 0.71.

We'll evaluate a convolution kernel and show that we can easily
match or outperform the deep learning baselines without too
much effort.

This was originally run on an GTX1070 GPU with 8 GB of RAM, using xGPR 0.1.3.0.

In [3]:
import os
import shutil
import subprocess
import math
import time
import zipfile

import pandas as pd
import numpy as np

from xGPR import xGPRegression as xGPReg
from xGPR import build_offline_sequence_dataset
from xGPR.static_layers import FastConv1d

In [2]:
#This may take a minute...
subprocess.run(["git", "clone", "https://github.com/J-SNACKKB/FLIP"])

shutil.move(os.path.join("FLIP", "splits", "aav", "full_data.csv.zip"), "full_data.csv.zip")
fname = "full_data.csv.zip"

with zipfile.ZipFile(fname, "r") as zip_ref:
    zip_ref.extractall()

os.remove("full_data.csv.zip")


shutil.rmtree("FLIP")

Cloning into 'FLIP'...
Checking out files: 100% (59/59), done.


In [4]:
raw_data = pd.read_csv("full_data.csv")
os.remove("full_data.csv")

/tmp/ipykernel_20174/4195622896.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("full_data.csv")


In [5]:
raw_data["input_seq"] = [f.upper().replace("*", "") for f in raw_data["mutated_region"].tolist()]

We'll use simple one-hot encoding for the sequences. This may take a minute to set up.

In [7]:
def one_hot_encode(input_seq_list, y_values, chunk_size, ftype = "train"):
    aas = ["A", "C", "D", "E", "F", "G", "H", "I",
               "K", "L", "M", "N", "P", "Q", "R", "S", "T",
               "V", "W", "Y", "-"]
    output_x, output_y = [], []
    xfiles, yfiles = [], []
    fcounter = 0
    
    for seq, y_value in zip(input_seq_list, y_values):
        encoded_x = np.zeros((1,57,21), dtype = np.uint8)
        for i, letter in enumerate(seq):
            encoded_x[0, i, aas.index(letter)] = 1

        output_x.append(encoded_x)
        output_y.append(y_value)

        if len(output_x) >= chunk_size:
            xfiles.append(f"{fcounter}_{ftype}_xblock.npy")
            yfiles.append(f"{fcounter}_{ftype}_yblock.npy")
            np.save(xfiles[-1], np.vstack(output_x))
            np.save(yfiles[-1], np.asarray(output_y))
            fcounter += 1
            output_x, output_y = [], []
            print(f"Encoded file {fcounter}")
    return xfiles, yfiles

In [8]:
train_data = raw_data[raw_data["des_mut_split"]=="train"]
test_data = raw_data[raw_data["des_mut_split"]=="test"]


train_x_files, train_y_files = one_hot_encode(train_data["input_seq"].tolist(),
                                              train_data["score"].tolist(), 2000, "train")
test_x_files, test_y_files = one_hot_encode(test_data["input_seq"].tolist(),
                                            test_data["score"].tolist(), 2000, "test")

Encoded file 1
Encoded file 2
Encoded file 3
Encoded file 4
Encoded file 5
Encoded file 6
Encoded file 7
Encoded file 8
Encoded file 9
Encoded file 10
Encoded file 11
Encoded file 12
Encoded file 13
Encoded file 14
Encoded file 15
Encoded file 16
Encoded file 17
Encoded file 18
Encoded file 19
Encoded file 20
Encoded file 21
Encoded file 22
Encoded file 23
Encoded file 24
Encoded file 25
Encoded file 26
Encoded file 27
Encoded file 28
Encoded file 29
Encoded file 30
Encoded file 31
Encoded file 32
Encoded file 33
Encoded file 34
Encoded file 35
Encoded file 36
Encoded file 37
Encoded file 38
Encoded file 39
Encoded file 40
Encoded file 41
Encoded file 42
Encoded file 43
Encoded file 44
Encoded file 45
Encoded file 46
Encoded file 47
Encoded file 48
Encoded file 49
Encoded file 50
Encoded file 51
Encoded file 52
Encoded file 53
Encoded file 54
Encoded file 55
Encoded file 56
Encoded file 57
Encoded file 58
Encoded file 59
Encoded file 60
Encoded file 61
Encoded file 62
Encoded file 63
E

In [9]:
#Notice that unlike for tabular data, we build an offline sequence dataset (not fixed vector).
#This dataset might be too large to load into memory, so we'll build an offline dataset only.
training_dset = build_offline_sequence_dataset(train_x_files, train_y_files, chunk_size = 2000,
                                              skip_safety_checks = False)

Here we'll evaluate FHTConv1d. Convolution kernels are usually slower than RBF / Matern, especially if the sequence is long. We'll tune using 3000 random features, which is closer to the upper end of what you want to use for
tuning with the ``crude_bayes`` function; in general, if you want to tune using a larger number of random features,
it's better to find a good starting point with a smaller number of RFFs, then use
``my_model.tune_hyperparams_fine_direct`` or ``my_model.tune_hyperparams_fine_bayes`` as illustrated in the tabular data example to "fine-tune" that result with a larger number of RFFs. Finally, we could alternatively use ``crude_grid`` (illustrated in the small molecule tutorial) or ``crude_lbfgs`` instead.

Note that we specify a kernel specific option in the kernel_specific_params dict -- the width of the convolution to use. Just as with a convolutional network, the width of the convolution filters can affect performance. One way to
choose a good setting: see what marginal likelihood score you get from hyperparameter tuning (e.g. with ``crude_bayes`` or ``crude_grid``) using a small number of RFFs (e.g. 1024 - 2048) for several different settings of "conv_width". The smallest score achieved likely corresponds to the best value for "conv_width".

In [10]:
aav_model = xGPReg(training_rffs = 3000, fitting_rffs = 8192, variance_rffs = 1024,
                  kernel_choice = "FHTConv1d",
                   kernel_specific_params = {"conv_width":9},
                   verbose = True, device = "gpu")

start_time = time.time()
hparams, niter, best_score, scores = aav_model.tune_hyperparams_crude_bayes(training_dset,
                                                                              max_bayes_iter = 30)
end_time = time.time()

print(f"Best estimated negative marginal log likelihood: {best_score}")
print(f"Wallclock: {end_time - start_time}")

starting_tuning
Grid point 0 acquired.
Grid point 1 acquired.
Grid point 2 acquired.
Grid point 3 acquired.
Grid point 4 acquired.
Grid point 5 acquired.
Grid point 6 acquired.
Grid point 7 acquired.
Grid point 8 acquired.
Grid point 9 acquired.
New hparams: [-0.8298276]
Additional acquisition 10.
New hparams: [-0.838633]
Additional acquisition 11.
New hparams: [-0.8383481]
Additional acquisition 12.
New hparams: [-0.8468422]
Additional acquisition 13.
New hparams: [-0.8321154]
Additional acquisition 14.
New hparams: [-0.8445669]
Additional acquisition 15.
New hparams: [-0.8072598]
Best score achieved: 121310.218
Best hyperparams: [-0.852417  -1.6094379 -0.8298276]
Tuning complete.
Best estimated negative marginal log likelihood: 121310.218
Wallclock: 667.0209712982178


Now we'll build a preconditioner and fit. In this case, since the
dataset is small and we're not using too many random features,
we're unlikely to have a disk space issue if we pre-generate
the random features, and for convolution kernels, especially on long sequences / graphs,
this is usually faster vs generating them on the fly (whereas for Matern / RBF / poly, generating
on the fly on GPU is faster).

In [11]:
pretransformed_data = aav_model.pretransform_data(training_dset, os.getcwd())

Now pretransforming data.


In [12]:
start_time = time.time()
preconditioner, ratio = aav_model.build_preconditioner(pretransformed_data, max_rank = 1024,
                                                      method = "srht_2")
end_time = time.time()
print(f"Wallclock: {end_time - start_time}")

Chunk 0 complete.
Chunk 10 complete.
Chunk 20 complete.
Chunk 30 complete.
Chunk 40 complete.
Chunk 50 complete.
Chunk 60 complete.
Chunk 70 complete.
Chunk 80 complete.
Chunk 90 complete.
Chunk 0 complete.
Chunk 10 complete.
Chunk 20 complete.
Chunk 30 complete.
Chunk 40 complete.
Chunk 50 complete.
Chunk 60 complete.
Chunk 70 complete.
Chunk 80 complete.
Chunk 90 complete.
Wallclock: 51.56280732154846


In [13]:
start_time = time.time()
aav_model.fit(pretransformed_data, preconditioner = preconditioner, mode = 'cg',
             max_iter = 500)
end_time = time.time()
print(f"Wallclock: {end_time - start_time}")

starting fitting
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Now performing variance calculations...
Fitting complete.
Wallclock: 65.16638588905334


In [14]:
start_time = time.time()
all_preds, ground_truth = [], []
for xfile, yfile in zip(test_x_files, test_y_files):
    x, y = np.load(xfile), np.load(yfile)
    ground_truth.append(y)
    preds = aav_model.predict(x, get_var = False)
    all_preds.append(preds)
    
all_preds, ground_truth = np.concatenate(all_preds), np.concatenate(ground_truth)
end_time = time.time()
print(f"Wallclock: {end_time - start_time}")

Wallclock: 9.972362279891968


In [15]:
from scipy.stats import spearmanr

spearmanr(all_preds, ground_truth)

SpearmanrResult(correlation=0.7664973351782205, pvalue=0.0)

Notice we're already at 0.766, outperforming a CNN for sequences also trained
on one-hot encoded input. It is of course possible
to use another representation (e.g. the output of a language model)
as the input to a GP. We discuss some other
possible representations and show that for some tasks using a language
model embedding can improve results (for this dataset, to Spearman's r
of about 0.8, which is SOTA as of July 2023).

In [16]:
training_dset.delete_dataset_files()
pretransformed_data.delete_dataset_files()

In [17]:
for testx, testy in zip(test_x_files, test_y_files):
    os.remove(testx)
    os.remove(testy)